In [ ]:
import numpy as np
import pandas as pd
import os
from shutil import copyfile
import pickle
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import colorlover as cl


import neuro_morpho_toolbox as nmt
%matplotlib inline
import pickle
pickle_in = open("/home/penglab/FeaCal/all_ns.pickle","rb")
[ns] = pickle.load(pickle_in)


In [ ]:
sf = ns.features['soma_features'].region
ct = sf["Region"].value_counts().sort_values(ascending=False)
_ = sns.countplot(y="Region", 
                  data=sf.loc[sf.Region.isin(ct.index[ct>10])],
                 )

In [ ]:
_ = ns.ReduceDimUMAP(feature_set="projection_features", n_neighbors=100)
_ = ns.FeatureScatter(["CellType", "Hemisphere"], map="UMAP")

In [ ]:
def id_to_name(self, region_ID):
        # region_name can be either Abbreviation (checked first) or description
        if region_ID in self.level.index.tolist():
            return self.level.loc[region_ID,'Abbreviation']
        else:
            print("Cannot find any regions with ID %s." % region_ID)

In [ ]:
import multiprocessing
import time
def Contour_Matrix(M_ori, M_contour, x_in, y_in, z_in):
    assert np.sum(M_contour)== 0
    assert M_contour.shape == M_ori.shape
    centerV =  M_ori[x_in,y_in, z_in]  
    assert M_ori.shape == nmt.annotation.array.shape
    x1 = M_ori[max(0,x_in-1),y_in,z_in]            # -1,0,0
    x2 = M_ori[min(M_ori.shape[0]-1,x_in+1),y_in,z_in]#+1,0,0
    y1 = M_ori[x_in,max(0,y_in-1),z_in]# 0,-1,0
    y2 = M_ori[x_in,min(M_ori.shape[1]-1,y_in+1),z_in] # 0,+1,0
    z1 = M_ori[x_in,y_in,max(0,z_in-1)]# 0,0,-1
    z2 = M_ori[x_in,y_in,min(M_ori.shape[2]-1,z_in+1)]# 0,0,+1
    if not all(v == centerV for v in [x1,x2,y1,y2,z1,z2]):
        M_contour[x_in,y_in,z_in] = centerV
    return M_contour.copy()

In [ ]:


corenum = 20
Regtype, Regcount = np.unique(nmt.annotation.array, return_counts=True) 

ContourDic = {}

start = time.perf_counter ()
start=time.time()
iterReg = 0
cores = corenum#multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=cores)
# array where the index is not 0
M_region = nmt.annotation.array.copy()
M_region = M_region!= iterReg  # a np.ndarray
M_origin = nmt.annotation.array.copy()
co_1,co_2,co_3 = np.where(nmt.annotation.array != 0) 
borderM = nmt.annotation.array.copy()
borderM[:,:,:] = 0
pool_list=[]
result_list=[]
for iter_COR in range(len(co_1)):
    pool_list.append(pool.apply_async(Contour_Matrix, (M_origin,borderM, co_1[iter_COR], co_2[iter_COR], co_3[iter_COR])))
    print('Loading progess: %.6f' % (iter_COR/len(co_1)))
result_list=[xx.get() for xx in pool_list]

pool.close()
pool.join()
elapsed = (time.time() - start)
print('Time needed to run the whole matrix is '+ str(elapsed))
ContourDic['ALL']= sum([xx for xx in  result_list])

In [ ]:
aaa = np.zeros(nmt.annotation.array.shape)

In [ ]:
aaa.shape

In [ ]:
import multiprocessing
import time
def Contour_Matrix(M_orix_in, y_in, z_in):
    M_contour = M_ori.copy()
    M_contour[:,:,:] = 0
    centerV =  M_ori[x_in,y_in, z_in]  
    assert M_ori.shape == nmt.annotation.array.shape
    x1 = M_ori[max(0,x_in-1),y_in,z_in]            # -1,0,0
    x2 = M_ori[min(M_ori.shape[0]-1,x_in+1),y_in,z_in]#+1,0,0
    y1 = M_ori[x_in,max(0,y_in-1),z_in]# 0,-1,0
    y2 = M_ori[x_in,min(M_ori.shape[1]-1,y_in+1),z_in] # 0,+1,0
    z1 = M_ori[x_in,y_in,max(0,z_in-1)]# 0,0,-1
    z2 = M_ori[x_in,y_in,min(M_ori.shape[2]-1,z_in+1)]# 0,0,+1
    if not all(v == centerV for v in [x1,x2,y1,y2,z1,z2]):
        M_contour[x_in,y_in,z_in] = 1
    return [x_in,y_in,z_in]

corenum = 12
Regtype, Regcount = np.unique(nmt.annotation.array, return_counts=True) 

ContourDic = {}

start = time.perf_counter ()
start=time.time()
iterReg = 0
cores = corenum#multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=cores)
# array where the index is not 0
M_region = nmt.annotation.array.copy()
M_region = M_region!= iterReg  # a np.ndarray
M_origin = nmt.annotation.array.copy()
co_1,co_2,co_3 = np.where(nmt.annotation.array != 0) 

pool_list=[]
result_list=[]
for iter_COR in range(len(co_1)):
    pool_list.append(pool.apply_async(Contour_Matrix, (M_origin, co_1[iter_COR], 
                                                       co_2[iter_COR], co_3[iter_COR])))
    print('Loading progess: %.6f' % (iter_COR/len(co_1)))
result_list=[xx.get() for xx in pool_list]

pool.close()
pool.join()
elapsed = (time.time() - start)
print('Time needed to run the whole matrix is '+ str(elapsed))
#ContourDic['ALL']= ([xx for xx in  result_list])

In [ ]:
M_origin[1,1,1]=5
M_ttt = M_origin.copy()
M_ttt[1,1,1]=14
M_origin[1,1,1]

In [ ]:


corenum = 12
Regtype, Regcount = np.unique(nmt.annotation.array, return_counts=True) 

ContourDic = {}

start = time.perf_counter ()
start=time.time()
iterReg = 0
cores = corenum#multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=cores)
# array where the index is not 0
M_region = nmt.annotation.array.copy()
M_region = M_region!= iterReg  # a np.ndarray
M_origin = nmt.annotation.array.copy()
co_1,co_2,co_3 = np.where(nmt.annotation.array != 0) 

In [ ]:
co_1,co_2,co_3 = np.where(nmt.annotation.array != 0)
len(co_1)

In [ ]:


hier_resultDF = pd.read_excel('/home/penglab/NMTcode/hier_resultDF.xlsx')
kmeans_resultDF = pd.read_excel('/home/penglab/NMTcode/kmeans_resultDF.xlsx')
dbscan_resultDF = pd.read_excel('/home/penglab/NMTcode/dbscan_resultDF.xlsx')
hdbscan_resultDF = pd.read_excel('/home/penglab/NMTcode/hdbscan_resultDF.xlsx')
snn_resultDF= pd.read_excel('/home/penglab/NMTcode/snn_resultDF.xlsx')


In [ ]:
x1 = np.arange(9).reshape((3, 3))
x2 = np.arange(9).reshape((3, 3))
np.dot(x1,x2)

### 6- nearest array

In [ ]:
M_mask = nmt.annotation.array.copy()
np.sum(M_mask)

In [ ]:
co_1[iter_pixel]+2

In [ ]:
M_mask[co_1[iter_pixel]-1:co_1[iter_pixel]+2,co_2[iter_pixel]-1:co_2[iter_pixel]+2,co_3[iter_pixel]-1:co_3[iter_pixel]+2] = 1

In [ ]:
        M_mask = nmt.annotation.array.copy()
        M_mask[:,:,:] = 0
        M_mask[max(0,55-1):min(M_mask.shape[0],55+1),
                   max(0,55-1):min(M_mask.shape[1],55+1),
               max(0,55-1):min(M_mask.shape[2],55+1)] = 1
        print(np.sum(np.sum(M_mask)))

In [ ]:
Regtype

### Single line

### Parallel processing

In [ ]:
Regtype, Regcount = np.unique(nmt.annotation.array, return_counts=True) 

ContourDic = {}
i_p=0
for iterReg in Regtype[1:]:
    M_region = nmt.annotation.array == iterReg  # a np.ndarray
    co_1,co_2,co_3 = np.where(nmt.annotation.array == iterReg) 
    borderM = nmt.annotation.array.copy()
    borderM[:,:,:] = 0
    for iter_pixel in range(len(co_1)):
        M_mask = nmt.annotation.array.copy()
        M_mask[:,:,:] = 0
        M_mask[max(0,co_1[iter_pixel]-1),co_2[iter_pixel],co_3[iter_pixel]]=1
        M_mask[min(nmt.annotation.array.shape[0]-1,co_1[iter_pixel]+1),co_2[iter_pixel],co_3[iter_pixel]]=1
        M_mask[co_1[iter_pixel],max(0,co_2[iter_pixel]-1),co_3[iter_pixel]]=1
        M_mask[co_1[iter_pixel],min(nmt.annotation.array.shape[1]-1,co_2[iter_pixel]+1),co_3[iter_pixel]]=1
        M_mask[co_1[iter_pixel],co_2[iter_pixel],max(0,co_3[iter_pixel]-1)]=1
        M_mask[co_1[iter_pixel],co_2[iter_pixel],min(nmt.annotation.array.shape[2]-1,co_3[iter_pixel]+1)]=1
        assert M_mask.shape == nmt.annotation.array.shape
        if np.sum(np.sum(np.multiply(M_region==0,M_mask)))>0:
            borderM[co_1[iter_pixel],co_2[iter_pixel],co_2[iter_pixel]]=1
        print('Loading progess inside region ' +str(iterReg)+' : %.5f' % (iter_pixel/len(co_1)))
    ContourDic[iterReg] = borderM
    print('Loading progess for all brain regions: %.2f' % (i_p/len(Regtype)))

In [ ]:
    for iter_pixel in range(len(co_1)):
        M_mask = nmt.annotation.array.copy()
        M_mask[:,:,:] = 0
        M_mask[max(0,co_1[iter_pixel]-1),co_2[iter_pixel],co_3[iter_pixel]]=1
        M_mask[min(nmt.annotation.array.shape[0]-1,co_1[iter_pixel]+1),co_2[iter_pixel],co_3[iter_pixel]]=1
        M_mask[co_1[iter_pixel],max(0,co_2[iter_pixel]-1),co_3[iter_pixel]]=1
        M_mask[co_1[iter_pixel],min(nmt.annotation.array.shape[1]-1,co_2[iter_pixel]+1),co_3[iter_pixel]]=1
        M_mask[co_1[iter_pixel],co_2[iter_pixel],max(0,co_3[iter_pixel]-1)]=1
        M_mask[co_1[iter_pixel],co_2[iter_pixel],min(nmt.annotation.array.shape[2]-1,co_3[iter_pixel]+1)]=1
        assert M_mask.shape == nmt.annotation.array.shape
        if np.sum(np.sum(np.multiply(M_reg==0,M_mask)))>0:
            M_contour[co_1[iter_pixel],co_2[iter_pixel],co_2[iter_pixel]]=1

In [ ]:
import multiprocessing
import time
def Contour_Matrix(M_reg, M_contour, co_1_iter, co_2_iter, co_3_iter):
    M_contour = nmt.annotation.array.copy()
    M_contour[:,:,:] = 0
    for iter_pixel in range(len(co_1)):
        M_mask = nmt.annotation.array.copy()
        M_mask[:,:,:] = 0
        M_mask[max(0,co_1[iter_pixel]-1),co_2[iter_pixel],co_3[iter_pixel]]=1
        M_mask[min(nmt.annotation.array.shape[0]-1,co_1[iter_pixel]+1),co_2[iter_pixel],co_3[iter_pixel]]=1
        M_mask[co_1[iter_pixel],max(0,co_2[iter_pixel]-1),co_3[iter_pixel]]=1
        M_mask[co_1[iter_pixel],min(nmt.annotation.array.shape[1]-1,co_2[iter_pixel]+1),co_3[iter_pixel]]=1
        M_mask[co_1[iter_pixel],co_2[iter_pixel],max(0,co_3[iter_pixel]-1)]=1
        M_mask[co_1[iter_pixel],co_2[iter_pixel],min(nmt.annotation.array.shape[2]-1,co_3[iter_pixel]+1)]=1
        assert M_mask.shape == nmt.annotation.array.shape
        if np.sum(np.sum(np.multiply(M_reg==0,M_mask)))>0:
            M_contour[co_1[iter_pixel],co_2[iter_pixel],co_2[iter_pixel]]=1
    return 

In [ ]:
import multiprocessing
import time
def Contour_Matrix(M_reg, M_contour, co_1_iter, co_2_iter, co_3_iter):
    M_contour = nmt.annotation.array.copy()
    M_contour[:,:,:] = 0
    for iter_pixel in range(len(co_1_iter)):
        print((co_1[iter_pixel],co_2[iter_pixel],co_2[iter_pixel]))
        M_mask = nmt.annotation.array.copy()
        M_mask[:,:,:] = 0
        assert M_mask.shape == nmt.annotation.array.shape
        M_mask[max(0,co_1[iter_pixel]-1),co_2[iter_pixel],co_3[iter_pixel]]=1
        M_mask[min(nmt.annotation.array.shape[0]-1,co_1[iter_pixel]+1),co_2[iter_pixel],co_3[iter_pixel]]=1
        M_mask[co_1[iter_pixel],max(0,co_2[iter_pixel]-1),co_3[iter_pixel]]=1
        M_mask[co_1[iter_pixel],min(nmt.annotation.array.shape[1]-1,co_2[iter_pixel]+1),co_3[iter_pixel]]=1
        M_mask[co_1[iter_pixel],co_2[iter_pixel],max(0,co_3[iter_pixel]-1)]=1
        M_mask[co_1[iter_pixel],co_2[iter_pixel],min(nmt.annotation.array.shape[2]-1,co_3[iter_pixel]+1)]=1
        assert M_mask.shape == nmt.annotation.array.shape
        if np.sum(np.sum(np.multiply(M_reg==0,M_mask)))>0:
            M_contour[co_1[iter_pixel],co_2[iter_pixel],co_2[iter_pixel]]=1
            #print((co_1[iter_pixel],co_2[iter_pixel],co_2[iter_pixel]))
    return 



In [ ]:
corenum = 12
Regtype, Regcount = np.unique(nmt.annotation.array, return_counts=True) 

ContourDic = {}

for iterReg in Regtype[::-1]:
    start = time.perf_counter ()
    start=time.time()
    cores = corenum#multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=cores)
    M_region = nmt.annotation.array != iterReg  # a np.ndarray
    co_1,co_2,co_3 = np.where(nmt.annotation.array == iterReg) 
    borderM = nmt.annotation.array.copy()
    borderM[:,:,:] = 0
    pool_list=[]
    result_list=[]
    pool_list.append(pool.apply_async(Contour_Matrix, (M_region,borderM, co_1, co_2, co_3)))
        # 这里不能 get， 会阻塞进程

    #pool.apply_async之后的语句都是阻塞执行的，
    #调用 result.get() 会等待上一个任务执行完之后才会分配下一个任务。
    #事实上，获取返回值的过程最好放在进程池回收之后进行，避免阻塞后面的语句。
    result_list=[xx.get() for xx in pool_list]
    print(sum([xx for xx in  result_list]))
    # 最后我们使用一下语句回收进程池:
    pool.close()
    pool.join()
    elapsed = (time.time() - start)
    print('Time needed to run region '+ str(iterReg)+ 'is '+ str(elapsed))
    ContourDic[iterReg]= sum([xx for xx in  result_list])

In [ ]:
import time
import multiprocessing
def test1(borderM, x,y,z):
    print(len(x))
    borderM[x,y,z]=1
    return(x*x,y*y,z*z)

corenum = 12

i_p=0
for iterReg in Regtype[1:]:
    start = time.perf_counter ()
    start=time.time()
    cores = corenum#multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=cores)
    M_region = nmt.annotation.array == iterReg  # a np.ndarray
    co_1,co_2,co_3 = np.where(nmt.annotation.array == iterReg) 
    co_1 = (np.arange(10))
    co_2 = (np.arange(5))
    co_3 = (np.arange(6))
    borderM = nmt.annotation.array.copy()
    borderM[:,:,:] = 0
    pool_list=[]
    result_list=[]
    pool_list.append(pool.apply_async(test1, (borderM,co_1, co_2, co_3)))
        # 这里不能 get， 会阻塞进程

    #pool.apply_async之后的语句都是阻塞执行的，
    #调用 result.get() 会等待上一个任务执行完之后才会分配下一个任务。
    #事实上，获取返回值的过程最好放在进程池回收之后进行，避免阻塞后面的语句。
    result_list=[xx.get() for xx in pool_list]
    print([xx for xx in  result_list])
    # 最后我们使用一下语句回收进程池:
    pool.close()
    pool.join()
    elapsed = (time.time() - start)
   

In [ ]:
np.arange(10)

In [ ]:
somaDF = ns.features['soma_features'].raw_data.copy()
for iter_idx in somaDF.index.tolist():
    ID_temp = nmt.annotation.array[somaDF.loc[iter_idx,'x'],somaDF.loc[iter_idx,'y'],somaDF.loc[iter_idx,'z']]
    M_region = nmt.annotation.array == ID_temp  # a np.ndarray
    M_mask = nmt.annotation.array.copy()
    M_mask[:,:,:] = 0
    mul_i = 1
    while np.sum(np.sum(np.multiply(M_region, M_mask)))==0:
        x_range_1 = somaDF.loc[iter_idx,'x']-5* mul_i
        x_range_2 = somaDF.loc[iter_idx,'x']+5* mul_i
        y_range_1 = somaDF.loc[iter_idx,'y']-5* mul_i
        y_range_2 = somaDF.loc[iter_idx,'y']+5* mul_i    
        z_range_1 = somaDF.loc[iter_idx,'z']-5* mul_i
        z_range_2 = somaDF.loc[iter_idx,'z']+5* mul_i  
        M_mask[x_range_1:x_range_2,:,:] = 0
        mul_i = mul_i +1

In [ ]:
# shape
print('Shape: ', nmt.annotation.array.shape)

# dtype
print('Datatype: ', nmt.annotation.array.dtype)

# size
print('Size: ', nmt.annotation.array.size)

# ndim
print('Num Dimensions: ', nmt.annotation.array.ndim)

In [ ]:
ns.features['soma_features'].raw_data



%matplotlib notebook
import mpld3
mpld3.enable_notebook()
import matplotlib.pyplot as plt
fig = plt.figure()
from mpl_toolkits.mplot3d import Axes3D
co_1,co_2,co_3 = np.where(nmt.annotation.array==9)
co_1 = co_1[co_3>=nmt.annotation.size['z']//4]
co_2 = co_2[co_3>=nmt.annotation.size['z']//4]
co_3 = co_3[co_3>=nmt.annotation.size['z']//4]
co_1 = co_1[co_3<=nmt.annotation.size['z']//2]
co_2 = co_2[co_3<=nmt.annotation.size['z']//2]
co_3 = co_3[co_3<=nmt.annotation.size['z']//2]
ax = fig.add_subplot(111, projection='3d')
ax.scatter(co_1, co_2, co_3, c='r', marker='o')


co_4,co_5,co_6 = np.where(nmt.annotation.array==0)

#choose_4 = np.sum((nmt.annotation.size['z']//4<=co_4)&(co_4<=nmt.annotation.size['z']//4))
#choose_5 = np.sum((min(co_2)<=co_5)&(co_5<=max(co_2)))
#choose_6 = np.sum((min(co_3)<=co_6)&(co_6<=max(co_3)))



    
ax.scatter(co_4[(nmt.annotation.size['z']//4<=co_4)&(co_4<=nmt.annotation.size['z']//2)], co_5[(nmt.annotation.size['z']//4<=co_4)&(co_4<=nmt.annotation.size['z']//2)], co_6[(nmt.annotation.size['z']//4<=co_4)&(co_4<=nmt.annotation.size['z']//2)], c='g', marker='o')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')


In [ ]:
type(co_1)

In [ ]:
co_1,co_2,co_3 = np.where(nmt.annotation.array==9)

In [ ]:
list9 = list(zip(co_1,co_2,co_3))
list9[0]

In [ ]:
co_1,co_2,co_3 = np.where(nmt.annotation.array==9)
co_1 = co_1[co_3<=nmt.annotation.size['z']//2]
co_2 = co_2[co_3<=nmt.annotation.size['z']//2]
co_3 = co_3[co_3<=nmt.annotation.size['z']//2]
ls_9x = []
ls_9y = []
ls_9z = []
ls_other= []
for x,y,z in zip(co_1,co_2,co_3):
    for i in range(-1,2):
        for j in range(-1,2):
            for k in range(-1,2):
                if nmt.annotation.array[x+i,y+j,z+k] !=9:
                    print(nmt.annotation.array[x+i,y+j,z+k])
                    ls_9x.append(x)
                    ls_9y.append(y)
                    ls_9z.append(z)
                    ls_other.append(nmt.annotation.array[x+i,y+j,z+k])


In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
fig = plt.figure()
from mpl_toolkits.mplot3d import Axes3D

ax = fig.add_subplot(111, projection='3d')

ax.scatter(ls_9x, ls_9y, ls_9z, c='g', marker='o')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

In [ ]:
IDtype, IDcount = np.unique(ls_other, return_counts=True) 

In [ ]:
IDtype

In [ ]:
list(zip(co_1,co_2,co_3))
ls_9 = []
ls_0 = []

for i in range(-1,2):
    for j in range(-1,2):
        for k in range(-1,2):
            if nmt.annotation.array[x+i,y+j,z+k] != nmt.annotation.array[x,y,z]:
                ls.append([x,y,z])


In [ ]:
co_2

In [ ]:
from scipy.spatial import distance
import ast
import time
#ns.features['soma_features'].raw_data
def metric_Cal(raw_data, regionINFO):
    start = time.time()
    raw_data = ns.features['soma_features'].raw_data
    somaDF = raw_data.copy()
    scaledDF = pd.DataFrame()
    scaledDF["x"] = (somaDF["x"] / nmt.annotation.space["x"]).copy()
    scaledDF["y"] = (somaDF["y"] / nmt.annotation.space["y"]).copy()
    scaledDF["z"] = (somaDF["z"] / nmt.annotation.space["z"]).copy()
    #flip the somalocation
    scaledDF["z"][scaledDF['z']>(nmt.annotation.size["z"]//2)] = scaledDF["z"][scaledDF["z"]>(nmt.annotation.size['z']//2)] -nmt.annotation.size["z"]//2 
    i_p = 0
    for idx in scaledDF.index.tolist():
        i_p = i_p+1
        tempCoor = np.array([[scaledDF.loc[idx,'x'],scaledDF.loc[idx,'y'],scaledDF.loc[idx,'z']]])
        tempID = nmt.annotation.array[int(scaledDF.loc[idx,'x']),int(scaledDF.loc[idx,'y']),int(scaledDF.loc[idx,'z'])]
        coords = []
        if regionINFO.loc[tempID ,'Vx']=='unknown' :
            scaledDF.loc[idx,'SqEuclidean'] = 'unknown'
            continue
        if regionINFO.loc[tempID ,'Vx']=='oob' :
            scaledDF.loc[idx,'SqEuclidean'] = 'oob'
            continue
        for icoor in range(len(ast.literal_eval(regionINFO.loc[tempID ,'Vx']))):
            coords.append([ast.literal_eval(regionINFO.loc[tempID ,'Vx'])[icoor],
                           ast.literal_eval(regionINFO.loc[tempID ,'Vy'])[icoor],
                                            ast.literal_eval(regionINFO.loc[tempID ,'Vz'])[icoor]])
        zs = np.sort(distance.cdist(tempCoor,coords, 'sqeuclidean'))[0,0:min(10,distance.cdist(tempCoor,coords, 'sqeuclidean').shape[1])]
        scaledDF.loc[idx,'SqEuclidean'] = str(zs.tolist())
        
        print('Load progress: %.5f'% (i_p/scaledDF.shape[0]))
    end = time.time()
    print("Total loading time: %.2f" % (end-start))
    return scaledDF.copy()

In [ ]:
for idx in scaled_result.index.tolist():
    

In [ ]:
scaled_result = metric_Cal(ns.features['soma_features'].raw_data, regionINFO)

In [ ]:
scaled_result.to_excel('/home/penglab/NMTcode/somaINFO.xlsx')

In [ ]:

for idx in scaled_result.index.tolist():
    templist = scaled_result.loc[idx,'SqEuclidean'].replace("[", "").replace("]", "").replace(" ","").split(',')
    scaled_result.loc[idx,'minDIS'] = templist[0]

In [ ]:
scaled_result['minDIS']

In [ ]:
import plotly 
import plotly.graph_objs as go

import scipy
y = scaled_result['minDIS'].astype(float).values.tolist()

fig = plotly.figure_factory.create_violin(y, title='Violin Plot', colors='#604d9e')
#py.iplot(fig, filename='alcohol-violin-visual')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



fig, axes = plt.subplots()

axes.violinplot(dataset = [scaled_result['minDIS'].astype(float).values] )

axes.set_title('Soma Loc v.s. Region Vertices distance')
axes.yaxis.grid(True)
axes.set_xlabel('')
axes.set_ylabel('min distance')

plt.show()

In [ ]:
nmt.annotation.array[230,13,54]

In [ ]:
np.mean(scaled_result['minDIS'].astype(float))

have a higher density at . That is very significant because as in the SepalLengthCm description, a mean value is at 9067.528044686285.

In [ ]:
import seaborn as sns
sns.set(style="dark")
ax = sns.violinplot(x=scaled_result['minDIS'].astype(float),palette="Set2")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull


# 8 points defining the cube corners
pts = np.array(coordi)#np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0],
                #[0, 0, 1], [1, 0, 1], [1, 1, 1], [0, 1, 1], ])

hull = ConvexHull(pts)

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

# Plot defining corner points
ax.plot(pts.T[0], pts.T[1], pts.T[2], "ko")

# 12 = 2 * 6 faces are the simplices (2 simplices per square face)


# Make axis label
for i in ["x", "y", "z"]:
    eval("ax.set_{:s}label('{:s}')".format(i, i))

plt.show()

In [ ]:
ns.features['soma_features'].raw_data

In [ ]:
from scipy.spatial import distance
import ast
#ns.features['soma_features'].raw_data
def metric_Cal(raw_data, regionINFO):
    somaDF = raw_data.copy()
    for idx in somaDF.index.tolist():
        tempCoor = np.array([(somaDF.loc[idx,'x'],somaDF.loc[idx,'y'],somaDF.loc[idx,'z'])])
        zs = np.sum(distance.cdist(tempCoor , ast.literal_eval(regionINFO.loc[idx,'Coor']), 'sqeuclidean'))
        somaDF.loc[idx,'SUMsqeuclidean'] = zs
    return somaDF.copy()